In [ ]:
import numpy as np
import numpy.linalg as la
import time

# Solving Ax=b

In [ ]:
n = 1000
A = np.random.randn(n,n)
x = np.random.rand(n)
b = A @ x

In [ ]:
t0 = time.time()
Ainv = la.inv(A) # use inversion
x2 = Ainv @ b
t1 = time.time()
print("{} sec.".format(t1 - t0))
la.norm(x - x2)

0.16173005104064941 sec.


2.2244454092147896e-11

Solving $Ax=b$ using solve function

In [ ]:
t0 = time.time()
x2 = la.solve(A, b) # use solve function
t1 = time.time()
print("{} sec.".format(t1 - t0))
la.norm(x - x2)

0.061837196350097656 sec.


1.5113818857765497e-12

# ScyPy linear Algebra

We’re now going to switch gears and start using scipy.linalg instead of numpy.linalg. From the user’s point of view, there isn’t really any difference, except scipy.linalg has all the same functions as numpy.linalg as well as additional functions.

In [ ]:
import scipy as sp
import scipy.linalg as sla

In [ ]:
n = 1000
A = np.random.randn(n,n)
x = np.random.rand(n)
b = A @ x

x1 = la.solve(A, b) # numpy
x2 = sla.solve(A, b) # scipy
la.norm(x1 - x2)

0.0

# LU Decomposition

The first type of factorization we'll look at is a $LU$ decomposition, where $L$ is lower-triangular and $U$ is upper triangular. For numerical stability, this is often computed with a pivoting strategy, which means there is also row or column permutation matrix $PA=PLU$




In [ ]:
n = 1000
A = np.random.randn(n, n)
x = np.random.rand(n)
b = A @ x

P, L, U = sla.lu(A)

In [ ]:
sla.norm(P @ L @ U - A)

2.5680294418182e-12

The nice thing about triangular matrices is that they can solve linear systems in $O(n^2)$ time, instead of $O(n^3)$ time for general matrices, using the forward or backward substitution algorithms. There is a special function solve_triangular for this reason:

In [ ]:
y = L @ x
x2 = sla.solve_triangular(L, y, lower=True)
sla.norm(x2 - x)

4.800821997353014e-13

In [ ]:
def my_solve(A, b):
    """
    solve A * x = b for x

    use LU decomposition
    """
    P, L, U =sla.lu(A)
    x = sla.solve_triangular(U,sla.solve_triangular(L,P.T @ b,lower=True),lower=False)
    return x

In [ ]:
b = A @ x
t0 = time.time()
x2 = sla.solve(A, b)
t1 = time.time()
print("{} sec.".format(t1 - t0))
print(sla.norm(x - x2))

0.12415003776550293 sec.
2.8641579540668785e-12


In [ ]:
t0 = time.time()
x2 = my_solve(A, b)
t1 = time.time()
print("{} sec.".format(t1 - t0))
print(sla.norm(x - x2))

0.06873130798339844 sec.
1.258590129902125e-10


In [ ]:
print(sla.norm(x - x2))

2.9682391283278526e-11


# QR Decomposition

The QR decomposition, $A=QR$, contains a matrix $Q$ with orthonormal columns, and an upper triangular matrix $R$. For stability reasons, column pivoting is often used which means there is often a permutation matrix $P$ and $A=QRP$.

In [ ]:
m = 1000
n = 500
A = np.random.randn(m, n)

Q, R = sla.qr(A, mode='economic') # no pivoting (not use la)
la.norm(Q @ R  - A)

5.403067620892784e-13

In [ ]:
Q.shape, R.shape

((1000, 500), (500, 500))

In [ ]:
Q, R, P = sla.qr(A, pivoting=True) # pivoting
la.norm((Q @ R)  - A[:,P])

6.183505446023941e-13

# Solving Least-squares problem

$$\min_{x}\|Ax-b\|$$

For full-rank (square) matrices, this is equivalent to solve.


In [ ]:
m = 1000
n = 500
A = np.random.randn(m, n)
x = np.random.rand(n)
b = A @ x

In [ ]:
t0 = time.time()
x2 = la.lstsq(A, b, rcond=None)[0]
t1 = time.time()
print("{} sec.".format(t1 - t0))
sla.norm(x - x2)

0.16371583938598633 sec.


4.605085589339936e-14

In [ ]:
t0 = time.time()
x3 = sla.lstsq(A, b)[0]
t1 = time.time()
print("{} sec.".format(t1 - t0))
sla.norm(x - x3)

0.1588444709777832 sec.


4.605085589339936e-14

In [ ]:
def my_lstsq(A, b):
    """
    least squares solution ||b - A*x||

    Uses QR decomposition
    """
    Q, R = sla.qr(A, mode='economic')
    x = sla.solve_triangular(R, Q.T @ b, lower=False)
    return x

In [ ]:
t0 = time.time()
x4 = my_lstsq(A, b)
t1 = time.time()
print("{} sec.".format(t1 - t0))
sla.norm(x - x4)

0.07330012321472168 sec.


1.3625205066392332e-14